In [ ]:
-- calculate top 5 customers with their revenue

CREATE OR REFRESH MATERIALIZED VIEW top_customers (
    customer_unique_id string,
    customer_city string,
    customer_state string,
    total_Revenue double,
CONSTRAINT valid_revenue EXPECT(total_Revenue is not null and total_Revenue > 0) ON VIOLATION DROP ROW
)
COMMENT "Aggregated data to find out top 5 customers based on thier revenue"
TBLPROPERTIES ("quality" = "gold")
as
with calc_rev as 
(
select  cust.customer_unique_id,customer_city, customer_state,
        sum(price+freight_value) as total_Revenue
from live.DimCustomers as cust
inner join live.FactOrders as ord
    on cust.customer_id = ord.customer_id
inner join live.FactOrderItems ord_itm
    on ord_itm.order_id = ord.order_id
group by cust.customer_unique_id,customer_city, customer_state
),
rank_cust as (
select *,dense_rank() over(partition by customer_unique_id order by total_Revenue desc ) as rn 
from calc_rev 
)
select  customer_unique_id,
        customer_city,
        customer_state,
        total_Revenue
from rank_cust where rn <6

In [ ]:
-- calculate order payment status

CREATE OR REFRESH MATERIALIZED VIEW order_installment_status(
    order_id string,
    order_price double,
    paid_value double,
    balance_to_be_paid double,
    CONSTRAINT valid_price EXPECT (order_price is not null order_price > 0) ON VIOLATION DROP ROW
)
COMMENT = "To calculate installments status of the orders"
TBLPROPERTIES ("quality" = "gold")
AS 
select  ord.order_id, 
        sum(price + freight_value) as order_price,
        sum(payment_value) as paid_value,
        sum(price + freight_value) - sum(payment_value) as balance_to_be_paid
FROM live.DimOrders ord 
inner join live.FactPayments pmts
    on ord.order_id = pmts.order_id
group by ord.order_id

In [ ]:
-- top 10 best selling products
CREATE OR REFRESH MATERIALIZED VIEW top_10_best_selling_products (
    product_id string,
    product_category_name string,
    product_value double,
    CONSTRAINT valid_prd_cat EXPECT (product_category_name is not null) ON VIOLATION DROP ROW,
    CONSTRAINT valid_prd_value EXPECT (product_value is not null and product_value > 0)
)
COMMENT = "finding top 10 best selling products based on product value"
TBLPROPERTIES ("quality" = "gold")
AS 
WITH calc_prod_val as 
(
SELECT  product_id,
        count(order_item_id) as no_of_times_purchased,
        sum(distinct (price+freight_value)) as product_value
FROM live.FactOrderItems
GROUP BY product_id
)
, rank_prd as 
(
    SELECT  *,
            dense_rank() over(order by no_of_times_purchased desc) as rn 
    FROM calc_prod_val
)
SELECT  c.product_id,
        p.product_category_name,
        c.product_value
FROM rank_prd c 
join live.DimProducts p 
    on c.product_id = c.product_id
WHERE rn <= 10

In [ ]:
-- display the no of orders for each order status

CREATE OR REFRESH MATERIALIZED VIEW no_of_order_per_status ()
(
    order_status string,
    no_of_orders long,
    CONSTRAINT valid_no_orders EXPECT (no_of_orders is not null and no_of_orders > 0)
)
COMMENT = "To calculate the no of orders for each order status"
TBLPROPERTIES ("quality" = "gold")
AS 
SELECT  order_status,
        count(order_id)
FROM live.FactOrders
GROUP BY order_status
